In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Import the dataset and make it indexed
df = pd.read_csv('./smartphone_cleaned_v5.csv')
df = df.sort_values(by='model',ascending=True)
df["SN"] = list(range(len(df.index)))
df=df.set_index('SN')

#Remove the NA values
df =df.ffill(axis=0)
#Remove the errors where no memory slot but the NA filling method has given storage
df['extended_upto'] = np.where(df['extended_memory_available']==0,0,df['extended_upto'])
df.fillna({'fast_charging':22},inplace=True) #ffill didn't work for iphone 11 and iphone 11 pro in the 1st run of code. so it manually for 2 mobiles
df.fillna({'extended_upto':0},inplace=True) #ffill didn't work for iphone 11 and iphone 11 pro in the 1st run of code. so it manually for 2 mobiles
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 980 entries, 0 to 979
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand_name                 980 non-null    object 
 1   model                      980 non-null    object 
 2   price                      980 non-null    int64  
 3   rating                     980 non-null    float64
 4   has_5g                     980 non-null    bool   
 5   has_nfc                    980 non-null    bool   
 6   has_ir_blaster             980 non-null    bool   
 7   processor_brand            980 non-null    object 
 8   num_cores                  980 non-null    float64
 9   processor_speed            980 non-null    float64
 10  battery_capacity           980 non-null    float64
 11  fast_charging_available    980 non-null    int64  
 12  fast_charging              980 non-null    float64
 13  ram_capacity               980 non-null    float64
 14 

In [3]:


# df3 = df[df.index.isin(df2.index)]
# df3[df3.index.isnull()]
# df4=df[df['extended_memory_available']==0]
# df4[df4['extended_upto'] != 0]


df.info()
# df[df['fast_charging'].null()]

<class 'pandas.core.frame.DataFrame'>
Index: 980 entries, 0 to 979
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand_name                 980 non-null    object 
 1   model                      980 non-null    object 
 2   price                      980 non-null    int64  
 3   rating                     980 non-null    float64
 4   has_5g                     980 non-null    bool   
 5   has_nfc                    980 non-null    bool   
 6   has_ir_blaster             980 non-null    bool   
 7   processor_brand            980 non-null    object 
 8   num_cores                  980 non-null    float64
 9   processor_speed            980 non-null    float64
 10  battery_capacity           980 non-null    float64
 11  fast_charging_available    980 non-null    int64  
 12  fast_charging              980 non-null    float64
 13  ram_capacity               980 non-null    float64
 14 

In [ ]:
from copy import deepcopy
df2 = deepcopy(df)


df2=df2.dropna(how='any',axis=0)
df2.info()

In [ ]:
category = 'processor_speed'
df[category].info()

In [ ]:
df['num_cores'].info()
df_t = df.isnull()
df_t.head()

In [7]:
list(df.loc[0:5,'model'])

['Apple iPhone 11',
 'Apple iPhone 11 (128GB)',
 'Apple iPhone 11 Pro Max',
 'Apple iPhone 12',
 'Apple iPhone 12 (128GB)',
 'Apple iPhone 12 (256GB)']

In [ ]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression.replace(" ", "").replace("x", "*")
        return eval(expression)
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    
    
def categorize_phone(row):
    condition_fast_processor = row["processor_speed"]*10/max_speed
    condition_fluid_display = 10 if row["refresh_rate"] >60 else 5
    condition_many_cores = row["num_cores"]*10/max_cores
    condition_nfc = 5 if row["has_nfc"]==True  else 0
    condition_5g=5 if row["has_5g"]==True else 0
    condition_resolution = (8 + 2*row['pixels']/max_resolution) if row['pixels']>=q2_resolution else 8*row['pixels']/q2_resolution
    total = condition_5g+condition_fluid_display +  condition_fast_processor + condition_nfc + condition_many_cores +condition_resolution
    if total > 40: return "flagship"
    elif total >30: return "midrange"
    else: return "average"
    # no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
    
def categorize_camera(row):
    if row["phone_category"] == "flagship":
        c1 = 10
    elif row["phone_category"] == "midrange":
        c1 = 8
    else:
        c1 = 5

    c2 = (8 + row["primary_camera_rear"]*2/max_rear) if row["primary_camera_rear"] > 12 else 8*row["primary_camera_rear"]/12
    c3 = (8 + row["primary_camera_rear"]*2/max_front) if row["primary_camera_rear"] > 10 else 8*row["primary_camera_rear"]/10
    
    brand_quality = {'apple':5, 'google':5, 'samsumg':4,'huawei':4}
    c4 = brand_quality[row['brand_name']] if row["brand_name"] in brand_quality else 3

    total = 10 + 10 + 10 + 5 # sum of maximum of c1 + c2 + c3 + c4
    rating = (c1+c2+c3)*10.0/total
    return rating
    
def categorize_performance(row):
    c1 = row["processor_speed"]*10/max_speed
    c2 = row["num_cores"]*10/max_cores
    c3 = (8 + row["ram_capacity"]*2/max_ram) if row["ram_capacity"] > 8 else row["ram_capacity"]/8  
    
    total = 10 + 10 + 10
    rating = (c1+c2+c3)*10.0/total
    return rating

In [ ]:
x = 8*8/12
x

In [ ]:
from copy import deepcopy
df2 = deepcopy(df)

#defining the ceiling
max_cores = max(df2["num_cores"].unique())
max_speed = max(df2["processor_speed"].unique())
max_ram = max(df2["ram_capacity"].unique())
max_price = max(df2["price"].unique())
max_rear = max(df2["primary_camera_rear"].unique())
max_front = max(df2["primary_camera_front"].unique())
max_resolution = max(df2["pixels"].unique())
max_refresh = max(df2["refresh_rate"].unique())

max_specs = [max_cores,max_speed,max_ram,max_price,max_rear,max_front,max_front,max_resolution,max_refresh]

#defining the ceilings
q1_cores = np.percentile(df['num_cores'].unique(),25)
q1_speed = np.percentile(df2["processor_speed"].unique(),25)
q1_ram = np.percentile(df2["ram_capacity"].unique(),25)
q1_price = np.percentile(df2["price"].unique(),25)
q1_rear = np.percentile(df2["primary_camera_rear"].unique(),25)
q1_front = np.percentile(df2["primary_camera_front"].unique(),25)
q1_resolution = np.percentile(df2["pixels"].unique(),25)
q1_refresh = np.percentile(df2["refresh_rate"].unique(),25)

q1_specs = [q1_cores,q1_speed,q1_ram,q1_price,q1_rear,q1_front,q1_front,q1_resolution,q1_refresh]

#defining the ceiling
q2_cores = np.percentile(df2["num_cores"].unique(),50)
q2_speed = np.percentile(df2["processor_speed"].unique(),50)
q2_ram = np.percentile(df2["ram_capacity"].unique(),50)
q2_price = np.percentile(df2["price"].unique(),50)
q2_rear = np.percentile(df2["primary_camera_rear"].unique(),50)
q2_front = np.percentile(df2["primary_camera_front"].unique(),50)
q2_resolution = np.percentile(df2["pixels"].unique(),50)
q2_refresh = np.percentile(df2["refresh_rate"].unique(),50)

q2_specs = [q2_cores,q2_speed,q2_ram,q2_price,q2_rear,q2_front,q2_front,q2_resolution,q2_refresh]

#defining the ceiling
q3_cores = np.percentile(df2["num_cores"].unique(),75)
q3_speed = np.percentile(df2["processor_speed"].unique(),75)
q3_ram = np.percentile(df2["ram_capacity"].unique(),75)
q3_price = np.percentile(df2["price"].unique(),75)
q3_rear = np.percentile(df2["primary_camera_rear"].unique(),75)
q3_front = np.percentile(df2["primary_camera_front"].unique(),75)
q3_resolution = np.percentile(df2["pixels"].unique(),75)
q3_refresh = np.percentile(df2["refresh_rate"].unique(),75)

q3_specs = [q3_cores,q3_speed,q3_ram,q3_price,q3_rear,q3_front,q3_front,q3_resolution,q3_refresh]



In [17]:
my_dictionary = {
    "name": "John",
    "age": 30,
    "city": "New York",
    "country": "USA",
    "occupation": "Engineer",
    '1':"Hello"
}
x = 1
my_dictionary[f'{x}']

'Hello'

In [16]:
keys = list(my_dictionary.keys())
values = list(my_dictionary.values())

list(zip(keys,values))

[('name', 'John'),
 ('age', 30),
 ('city', 'New York'),
 ('country', 'USA'),
 ('occupation', 'Engineer')]

In [ ]:
# condition_fast_processor = df["processor_speed"]>3
# condition_fluid_display = df["refresh_rate"]>60
# condition_many_cores = df["num_cores"]>=8
# condition_nfc = df["has_nfc"]==True
# condition_5g=df["has_5g"]==True
# condition_resolution = df["resolution"].apply(evaluate_resolution)
# no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
# # all_conditions = condition_fluid_display &  condition_fast_processor & condition_nfc & condition_5g & condition_many_cores & condition_resolution
# # most_conditions = 
# # average_conditions = 
# # better_than_no_conditions = 
# # no_conditions = 
# conditions = no_of_conditions_satisfied == 6

In [ ]:
from copy import deepcopy
df2 = deepcopy(df)
df2.head()
final_rec = deepcopy(df)
final_rec.head()


In [ ]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression['resolution'].replace(" ", "").replace("x", "*")
        return  eval(expression)
        
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    # return expression['resolution']
    
df['pixels'] = df.apply(evaluate_resolution,axis=1)
np.percentile(df['pixels'],99)

In [ ]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression.replace(" ", "").replace("x", "*")
        return eval(expression)
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False

max_cores = max(df["num_cores"].unique())
max_speed = max(df["processor_speed"].unique())
max_price = max(df["price"].unique())
max_rear = max(df["primary_camera_rear"].unique())
max_front = max(df["primary_camera_front"].unique())
max_resolution = max(df["pixels"].unique())
max_refresh = max(df["refresh_rate"].unique())

max_specs = [max_cores,max_speed,max_price,max_rear,max_front,max_front,max_resolution,max_refresh]
max_specs

In [ ]:
item = 'price'
p1 = np.percentile(df[item],1)
p2 = np.percentile(df[item],10)
p3 = np.percentile(df[item],90)
p4 = np.percentile(df[item],99)
p1,p2,p3,p4


In [ ]:
# pre-processing

#price
category = 'primary_camera_rear'
p1 = np.percentile(df[category],10)
p2 = np.percentile(df[category],90)
df2 = df.query(f"{category} >= {p1} and {category} <= {p2}")
max(df2[category])
# min(df[category].unique())


In [ ]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression['resolution'].replace(" ", "").replace("x", "*")
        return  eval(expression)
        
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    # return expression['resolution']
    
df2['pixels'] = df2.apply(evaluate_resolution,axis=1)
np.percentile(df2['pixels'],99)

In [ ]:
sorted(df2['pixels'].unique(),reverse=True)

In [ ]:
expensives = df.query("price <= 5449")
expensives.sort_values(by='price', ascending=True)

In [ ]:
objects = df[df["rating"].isnull()]
# print(objects)


from copy import deepcopy
df2 = deepcopy(df)
df2.head()

# max_cores = max(df["num_cores"].unique())
# max_speed = max(df["processor_speed"].unique())

# # print(max_cores,max_speed)

# # Assuming df is your DataFrame
df2['phone_category'] = df2.apply(categorize_phone, axis=1)
# df2['camera_category'] = df2.apply(categorize_camera, axis=1)

# objects = df2[df["rating"].isnull()]
# objects



# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

df2.head()


In [ ]:
final_rec = final_rec.query("brand_name == 'apple'")
final_rec.head()

In [ ]:
final_rec = final_rec.query("price >= 100000")
final_rec.head()

In [ ]:
final_rec = final_rec.query("refresh_rate <= 90")
final_rec.head()

In [ ]:
pixels = df.query("brand_name == 'google'")


In [ ]:
level5 = ['excellent', 'outstanding','exceptional','remarkable','first-rate','top-notch','exceptionally good',' splendid', 'impressive',"Superlative","Exemplary","Prime","Stellar","Astonishing","Fabulous","Phenomenal","Spectacular","Marvelous","Exceptionally","Incredible","Brilliant","Sublime","Distinguished","Peerless","Unparalleled","Eminent","Unsurpassed","Unbeatable","Epic"]
level1 = adjectives_worst = ["Abysmal","Appalling","Atrocious","Awful","Dreadful","Terrible","Horrendous","Horrible","Deplorable","Disastrous","Disgusting","Execrable","Lousy","Pathetic","Shameful","Unspeakable","Vile","Wretched","Miserable","Catastrophic","Grim","Woeful","Dire","Repulsive","Rotten","Detestable","Unacceptable","Inferior","Foul","Unsatisfactory"]
categories = [] #performance,camera,software,build,display,photo,video,video, storage,speaker

In [ ]:
stage1 = [
    'How do you want to get recommendation?',
    'Entering your own phrase',
    'Get your questions asked'
]

exact_questions = [
    '1.) IPhone 2.) Android 3.)Open to both',
    "Do you want a particular smartphone brand? \n1.) No 2.) Yes (Input your brand)",
    'What screen size would you prefer? \n1.) below 6.1" 2.) between 6.1" & 6.6" 3.) above 6.6" 4.) Doesn\'t matter',
    'Do you want your phone to have 5g? \n1.) Yes 2.) No 3.) Any',
    'What is your budget range Example (inside the "") "45000-50000"?'
]
performance_questions = [
    'Do you frequently play games on your smartphone? (y/n)',
    'Do you use a  lot of apps? (y/n)',
    'Do you want to take high quality photos and videos (y/n)',
    'Do you wish to use your smartphone for over 5 years? (y/n)',
    'Do you wish to have all new features in your smartphone? (y/n)'
]
camera_questions = [
    'Do you want to take high quality photos and videos? (y/n)',
    'Do you want a camera that shoot good in night time? (y/n)'
    'Would you love to shoot zoom and potrait shots? (y/n)'
    'Do you want to shoot many videos? (y/n)',
    'Do you want to shoot many photos? (y/n)'
]
software_questions = [
    'Do you want to get new updates for 5+ years? (y/n)'
    "Do you mind useless apps pre-installed in your smartphone? (y/n)",
    'Do you like to have easy software support? (y/n)'
    'Can you distinguish between smooth and non smooth animation? (y/n)'
]
display_questions = [
    'Do you do a lot of scrolling? (y/n)',
    "Do you like watching movies in mobile? (y/n)",
    'Do you consume a lot of media like youtube and more? (y/n)',
    'Do you wish to use phone a lot on sunlight? (y/n)'
]
storage_questions = [
    "Do you take a lot of photos and videos? (y/n)",
    'Do you store a lot of offline medias like movies or other files in smartphones? (y/n)',
    "Do you want your smartphone to be able to store everything you want for years to come? (y/n)",
    'Do you want to transfer from and to your smartphones quickly? (y/n)'
    'Do you download a lot of apps orr games? (y/n)',
]
speaker_questions = [
    "Would you like a good speaker? \n1.) yes (y) 2.) don't mind (n)"
]
battery_questions = [
    "Do you like a bigger battery? (y/n)",
    'Do you charge very rarely? (y/n)'
    "Do you waipcnt fast charging? (y/n)",
    'Do you want wireless charging? (y/n)',
]

all_categories = [exact_questions,performance_questions, camera_questions,software_questions,display_questions,storage_questions,speaker_questions]

In [ ]:
for prompts in stage1:
    print(prompts)
lst = []

In [ ]:
lst = []
for category in all_categories:
    for questions in category:
        lst.append(input(questions))


In [ ]:
df2.head()

In [ ]:
lst = []
lst.append('y')
lst.append('Hello')
lst.append('y')
# lst.append(True)
mapping_dict = {'y':True,'n':False}
lst = list(map(lambda x: mapping_dict[x] if x in mapping_dict else x,lst))
lst
# if any(lst):
#     print("HelloWOrld")

In [ ]:
string = '2000-3000'
low,high = map(int, string.split('-'))
low,high

In [ ]:
import pandas as pd

# Sample DataFrame
data = {'fruit': ['apple', 'banana', 'orange', 'mango', 'grape'],
        'color': ['red', 'yellow', 'orange', 'yellow', 'purple'],
        'price': [1.25, 0.75, 1.50, 1.00, 0.50]}
df = pd.DataFrame(data)
print(df)
print()
# Apply a discount to orange fruits
discounted_price = df[df['fruit'] == 'orange']['price'] * 0.9  # Vectorized operation

# Update the 'price' column for oranges
df.loc[df['fruit'] == 'orange', 'price'] = discounted_price

print(df)


In [ ]:
df

In [ ]:
def bad_software(row):
    if row['brand_name'] not in ['itel','samsung']:
        return row['rating']*0.5
    else: 
        return row['rating']
df['rating']= df.apply(bad_software,axis=1)
df

In [ ]:
lst1= [True, True, False]
lst2 = False
# any([False].append([False]))
([False for i in range(5)]+lst1).count(True)